# Introduction to `tcollections`

In [1]:
import json
import sys
sys.path.append('../src')
import tcollections
from tcollections import tlist, group

## Collection Types

In [2]:
tl = tlist(range(10))
tl

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [3]:
tl + tl

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
tl.map(lambda x: x*2)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [5]:
tl.sort(reverse=True)

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [6]:
tl.filter(lambda x: x % 2 == 0)

[0, 2, 4, 6, 8]

In [7]:
(tl*4).value_counts()

Counter({0: 4, 1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4, 9: 4})

In [ ]:
tl.reduce(lambda a,b: a+b, start=0), sum(tl)

45

In [9]:
tl.copy()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [27]:
tl.group.by(lambda x: x % 2)

Groups({0: [0, 2, 4, 6, 8], 1: [1, 3, 5, 7, 9]})

In [28]:
tl.group.multi(lambda x: (x % 2 == 0, x % 3 == 0))

NestedGroups({True: NestedGroups({True: [0, 6], False: [2, 4, 8]}), False: NestedGroups({False: [1, 5, 7], True: [3, 9]})})

## Grouping Items
This section concerns grouping elements.

### Grouping Functions

To group elements of a collection according to a single key, you can use either `groupby` or `group.by`.

In [11]:
grps = tcollections.group.by(tl, lambda x: x % 2)
grps

Groups({0: [0, 2, 4, 6, 8], 1: [1, 3, 5, 7, 9]})

In [12]:
grps = tl.group.by(lambda x: x % 2)
grps

Groups({0: [0, 2, 4, 6, 8], 1: [1, 3, 5, 7, 9]})

In [13]:
grps = tcollections.groupby(tl, lambda x: x % 2)
grps

Groups({0: [0, 2, 4, 6, 8], 1: [1, 3, 5, 7, 9]})

Use `groupby_multi` or `group.multi` if you need nested groups.

In [14]:
grps = tcollections.groupby_multi(tl, lambda x: (x % 2 == 0,x % 3 == 0))
print(grps.to_json(indent=4))

{
    "True": {
        "True": [
            0,
            6
        ],
        "False": [
            2,
            4,
            8
        ]
    },
    "False": {
        "False": [
            1,
            5,
            7
        ],
        "True": [
            3,
            9
        ]
    }
}


In [15]:
grps = tl.group.multi(lambda x: (x % 2 == 0,x % 3 == 0))
print(grps.to_json(indent=4))

{
    "True": {
        "True": [
            0,
            6
        ],
        "False": [
            2,
            4,
            8
        ]
    },
    "False": {
        "False": [
            1,
            5,
            7
        ],
        "True": [
            3,
            9
        ]
    }
}


In [16]:
grps = tcollections.group.multi(tl, lambda x: (x % 2,))
grps

NestedGroups({0: [0, 2, 4, 6, 8], 1: [1, 3, 5, 7, 9]})

In [17]:
grps = tcollections.groupby_multi(tl, lambda x: (x % 2,))
print(grps.to_json())

{"0": [0, 2, 4, 6, 8], "1": [1, 3, 5, 7, 9]}


### Aggregating Groups

In [18]:
agged = tl.group.by(lambda x: x % 2).agg(lambda x: sum(x))
print(json.dumps(agged, indent=2))

{
  "0": 20,
  "1": 25
}


In [19]:
tl.group.multi(lambda x: (x % 2 == 0,x % 3 == 0)).agg(lambda x: sum(x))

{True: {True: 6, False: 14}, False: {False: 13, True: 12}}

In [20]:
tl.group.multi(lambda x: (x % 2 == 0,x % 3 == 0)).agg(lambda x: sum(x))

{True: {True: 6, False: 14}, False: {False: 13, True: 12}}

In [21]:
tl.group.multi(lambda x: (x % 2 == 0,x % 3 == 0)).agg(lambda x: sum(x))

{True: {True: 6, False: 14}, False: {False: 13, True: 12}}

### Flattening and Ungrouping

In [22]:
grps = tl.group.by(lambda x: x % 2)
grps.ungroup()

[0, 2, 4, 6, 8, 1, 3, 5, 7, 9]

In [23]:
grps = tl.group.multi(lambda x: (x % 2 == 0,x % 3 == 0))
grps.ungroup()

[0, 6, 2, 4, 8, 1, 5, 7, 3, 9]

In [24]:
grps = tl.group.multi(lambda x: (x % 2 == 0,x % 3 == 0))
flat_grps = grps.flatten()
print(flat_grps.to_json(indent=4))

{
    "(True, True)": [
        0,
        6
    ],
    "(True, False)": [
        2,
        4,
        8
    ],
    "(False, False)": [
        1,
        5,
        7
    ],
    "(False, True)": [
        3,
        9
    ]
}
